# Dependency Handling

In [1]:
# Install pytorch

!pip3 install torch torchvision torchaudio
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [2]:
# Install Transformers (using for NLP model - allows for easily import model)
# Install Requests (allow requests to webpages for analysis)
# Install BeautifulSoup (extract relevant data from webpage)
# Install Pandas and Numpy (structure data)

!pip3 install requests transformers beautifulsoup4 pandas numpy

In [3]:
# Tokenizer - Parse through string to get numbers.
# AutoModelForSequenceClassification - Give us architecture from transformers to load in NLP model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

C:\Users\dawoo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setup Model (Initialization)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Encoding and Sentiment Calculation

In [5]:
tokens0 = tokenizer.encode('I am having a horrible day', return_tensors='pt') 
tokens1 = tokenizer.encode('خوش', return_tensors='pt') # Means "happy" 
tokens2 = tokenizer.encode('اِک شخص بددُعا سی کر گیا مُجھ پر, وُہ عہدِ ترقِ وفَاسِی کرگیا مُجھ پر', return_tensors='pt') # Negative sentence


In [6]:
# Debug

# tokens
# tokens[0]
# tokenizer.decode(tokens[0])

In [7]:
result0 = model(tokens0)
result1 = model(tokens1)
result2 = model(tokens2)

In [8]:
print(result0)
print(result1)
print(result2)

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.3005,  1.4749, -0.0459, -2.0774, -2.0999]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-1.4725, -1.1947,  0.2650,  1.0080,  1.1522]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0923, -0.5614, -0.0990, -0.1329,  0.6934]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [9]:
star_rating0 = int(torch.argmax(result0.logits)) + 1
star_rating1 = int(torch.argmax(result1.logits)) + 1
star_rating2 = int(torch.argmax(result2.logits)) + 1

print('Star Rating (result0): ', star_rating0)
print('Star Rating (result1): ', star_rating1)
print('Star Rating (result2): ', star_rating2)


Star Rating (result0):  1
Star Rating (result1):  5
Star Rating (result2):  5


#  Parse Website

In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}
req_sad = requests.get('https://hamariweb.com/poetries/sad_poetries2.aspx', headers=headers)
req_funny = requests.get('https://hamariweb.com/poetries/funny_poetries4.aspx', headers=headers)

soup_sad = BeautifulSoup(req_sad.text, 'html.parser')
soup_funny = BeautifulSoup(req_funny.text, 'html.parser')

regex_sad = re.compile('UrduText')
regex_funny = re.compile('UrduText')

result_website_sad = soup_sad.find_all('a', {'class':regex_sad})
result_website_funny = soup_funny.find_all('a', {'class':regex_funny})

reviews_sad = [i.text for i in result_website_sad]
reviews_funny = [i.text for i in result_website_funny]

# Debug
#reviews_sad
#reviews_funny


# Load Website Data into Dataframe and Get Star Rating

In [11]:
df_sad = pd.DataFrame(np.array(reviews_sad), columns=['string'])
df_funny = pd.DataFrame(np.array(reviews_funny), columns=['string'])

# Debug
#soup
#result_website
#reviews
#df['string'].iloc[1]

In [12]:
def StringToScore(string):
    tokens = tokenizer.encode(string, return_tensors='pt')
    output = model(tokens)
    return int(torch.argmax(output.logits)) + 1
    

In [13]:
# Debug
#StringToScore(df_sad['string'].iloc[0])

In [14]:
df_sad['sentiment'] = df_sad['string'].apply(lambda x: StringToScore(x[:15]))
#df_sad['string'] = df_sad['string'].replace('\n','')
df_funny['sentiment'] = df_funny['string'].apply(lambda x: StringToScore(x[:15]))

In [15]:
df_sad

,string,sentiment
0,\nتمہاری منتظر یوں تو ہزاروں گھر بناتی ہوںوہ ر...,1
1,\nہم انہیں وہ ہمیں بھلا بیٹھےدو گنہ گار زہر کھ...,5
2,\nخنجر سے کرو بات نہ تلوار سے پوچھومیں قتل ہوا...,3
3,\nوہ شخص حسن اپنا دکھا کر چلا گیادیوانہ مجھ کو...,5
4,اے نگاہ دوست یہ کیا ہو گیا کیا کر دیاپہلے پہلے...,5
5,رواں دواں ہے زندگی چراغ کے بغیر بھیہے میرے گھر...,3
6,وہ ہنس ہنس کے وعدے کیے جا رہے ہیںفریب تمنا دیے...,5
7,چشم نم پر مسکرا کر چل دیئےآگ پانی میں لگا کر چ...,1
8,چاہت کی اذیت رہائی نہ دے گیکہ بچنےکی راہ بھی د...,5
9,کون جیتا کون ہارا یہ کہانی پھر سہیہوگا کیسے اب...,3


In [16]:
df_funny

,string,sentiment
0,دھوبی کا کتا ہوں گھر کا نہ گھاٹ کابیگم بارہا ک...,3
1,کبھی دادی لگتی ھے کبھی نانی لگتی ھے ماڈل ھے پ...,1
2,ابتدا عشق میں یوں ہوا دل خراب آدھاکے جیسے سیکھ...,3
3,"وہ جب ناراض ہوتے ہیں تو ""کانپیں ٹانگ"" جاتی ہیں...",1
4,اتنا دبلا ہو گیا ہوں صنم تیری جدائی سےکھٹمل بھ...,1
5,ہم نے مانا کہ رپلائی نہ کرو گے تم لیکنٹرائی کر...,5
6,جس بھی شادی میں جاتا ہوںکسی نہ کسی لڑکی سے پیا...,5
7,اگر تمہارے وعدے نہ ہوتے کچےتو اب ہوتے ہمارے دو...,3
8,یہ اتوار نہیں آساں بس اِتنا ہی سمجھ لیجئےاِک ک...,1
9,گھر میں اگر کوئی عقل کی بات کردوتو سب یہی پوچھ...,3


In [17]:
print("Average Sentiment (Sad Poetry): ", df_sad['sentiment'].mean())

Average Sentiment (Sad Poetry):  3.6666666666666665


In [18]:
print("Average Sentiment (Funny Poetry): ", df_funny['sentiment'].mean())

Average Sentiment (Funny Poetry):  3.2


In [19]:
np.savetxt(r'text_sad.txt', df_sad['string'].values, fmt='%s', encoding='utf-8')
np.savetxt(r'text_funny.txt', df_funny['string'].values, fmt='%s', encoding='utf-8')